In [ ]:
import os
import json 
import librosa
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model, Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras import models, layers

import shutil

In [ ]:
train_meta = pd.read_csv("/kaggle/input/birdclef-2022/train_metadata.csv")

with open('/kaggle/input/birdclef-2022/scored_birds.json') as f:
    scored_birds = json.load(f)

train_meta

In [ ]:
labels = list(train_meta['primary_label'].unique())

In [ ]:
import soundfile as sf
import os

In [ ]:
def cutAudio(file_path, is_save):
    # First load the file
    filename = file_path.replace("/", "_")
    file_path = "/kaggle/input/birdclef-2022/train_audio/" + file_path
    audio, sr = librosa.load(file_path)

    buffer = 5 * sr

    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    audio_split = []
    audio_filenames = []
    
    while samples_wrote < samples_total: 
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        block = audio[samples_wrote : (samples_wrote + buffer)]
        audio_split.append(block)

        if is_save == True:
            out_filename = "/kaggle/working/each5s/split_" + str(counter) + "_" + filename
            audio_filenames.append(out_filename)
            sf.write(out_filename, block, sr)
        counter += 1
        samples_wrote += buffer
    return audio_split, sr, audio_filenames

In [ ]:
def splitTrainAudio(_df):
    data = []
    for index, row in _df.iterrows():
        cutAudio(row["filename"], True)
        audio_lst, sr, filenames = cutAudio(row["filename"], True)
        for idx, y in enumerate(audio_lst):
            data.append([row["primary_label"], row["filename"], filenames[idx]])

    data_df = pd.DataFrame(data, columns=['primary_label', 'original_filename', 'filename'])
    data_df.to_csv("/kaggle/working/data_df.csv", index=False)

In [ ]:
data_frames = []

for label in scored_birds:
    tmp_df = train_meta[train_meta["primary_label"] == label].reset_index(drop=True)
    data_frames.append(tmp_df)
    
sample_df = pd.concat(data_frames).reset_index(drop=True)
sample_df

In [ ]:
!mkdir -p "/kaggle/working/each5s"
splitTrainAudio(sample_df)

In [ ]:
data_df = pd.read_csv("/kaggle/working/data_df.csv")
data_df

In [ ]:
num_rows = 64
num_columns = 216
num_channels = 1
n_mels = 64

def extractFeatures(y, sr):
    feat = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=num_rows, n_mels=n_mels)
    if feat.shape[1] <= num_columns:
        pad_width = num_columns - feat.shape[1]
        feat = np.pad(feat, pad_width=((0,0),(0,pad_width)), mode='constant')
    return feat

In [ ]:
class DataGenerator(Sequence):
    def __init__(self,
                _X,
                batch_size=16,
                n_channels=1,
                n_columns=213,
                n_rows=128,
                shuffle=True):
        self.batch_size = batch_size
        self.X = _X
        self.n_channels = n_channels
        self.n_columns = n_columns
        self.n_rows = n_rows
        self.shuffle = shuffle
        self.img_indexes = np.arange(len(self.X))
        self.on_epoch_end()
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.img_indexes) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temps = [self.img_indexes[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temps)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.X))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temps):
        X = np.empty((self.batch_size, self.n_rows, self.n_columns))
        y = np.empty((self.batch_size), dtype=int)
        for i, ID in enumerate(list_IDs_temps):
            file_path = self.X.iloc[ID]["filename"]
            audio, sr = librosa.load(file_path)
            feat = extractFeatures(audio, sr)
            x_features = feat.tolist()
            label = self.X.iloc[ID]["target"]
            X[i] = np.array(x_features)
            y[i] = label
        X = X.reshape(X.shape[0], self.n_rows, self.n_columns, self.n_channels)
        
        
        
        return X, to_categorical(y, num_classes=len(labels))

In [ ]:
params = dict(batch_size=16,n_rows=num_rows,n_columns=num_columns,n_channels=num_channels,)

params_train = dict(shuffle=True,**params)

params_valid = dict(shuffle=False,**params)

In [ ]:
import matplotlib.pyplot as plt

def plot_his(history):
    plt.figure(1, figsize = (15,8))
    plt.subplot(221)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'])
    plt.subplot(222)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'])
    plt.show()

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Conv1D, Resizing
from keras import regularizers, optimizers
import pandas as pd
from tensorflow.keras import Input
import numpy as np

In [ ]:
input_shape=(64,64,1)

CNNmodel = Sequential()

CNNmodel.add(Input(shape=input_shape))
CNNmodel.add(Resizing(32, 32))

CNNmodel.add(Activation('relu'))
CNNmodel.add(Conv2D(32, (3, 3)))
CNNmodel.add(Activation('relu'))
CNNmodel.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel.add(Dropout(0.25))
CNNmodel.add(Conv2D(64, (3, 3), padding='same'))
CNNmodel.add(Activation('relu'))
CNNmodel.add(Conv2D(64, (3, 3)))
CNNmodel.add(Activation('relu'))
CNNmodel.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel.add(Dropout(0.5))
CNNmodel.add(Conv2D(128, (3, 3), padding='same'))
CNNmodel.add(Activation('relu'))
CNNmodel.add(Conv2D(128, (3, 3)))
CNNmodel.add(Activation('relu'))
CNNmodel.add(MaxPooling2D(pool_size=(2, 2)))
CNNmodel.add(Dropout(0.5))
CNNmodel.add(Flatten())
CNNmodel.add(Dense(512))
CNNmodel.add(Activation('relu'))
CNNmodel.add(Dropout(0.5))
CNNmodel.add(Dense(152, activation='softmax'))

CNNmodel.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

CNNmodel.summary()

In [ ]:
data_df['target'] = data_df['primary_label'].apply(lambda x: labels.index(x))
data_df

In [ ]:
def train_model(model, train_gen, val_gen):
    checkpoint_model_path = "/kaggle/working/mobilnetv2.h5"
    metric = "val_accuracy"
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=["accuracy"])
    
    num_epochs = 10
    checkpointer = ModelCheckpoint(
        filepath=checkpoint_model_path,
        monitor=metric, verbose=1, save_best_only=True)
    es_callback = EarlyStopping(monitor=metric, patience=5, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor=metric, factor=0.3, patience=1, verbose=1, min_delta=0.0001, cooldown=1, min_lr=0.00001)

    history = model.fit(train_gen,epochs=num_epochs,validation_data=val_gen,callbacks=[checkpointer,es_callback,reduce_lr],verbose=1)

    plot_his(history)

In [ ]:
X_train, X_valid, _, _ = train_test_split(data_df, data_df["target"], test_size=0.1, random_state=42)
train_generator = DataGenerator(X_train, **params_train)
valid_generator = DataGenerator(X_valid, **params_valid)

In [ ]:
cnn_model = CNNmodel

In [ ]:
train_model(cnn_model, train_generator, valid_generator)

In [ ]:
import time

In [ ]:
test_path = "/kaggle/input/birdclef-2022/test_soundscapes/"
files = [f.split('.')[0] for f in sorted(os.listdir(test_path))]

In [ ]:
cnn_model.load_weights("/kaggle/working/mobilnetv2.h5")

In [ ]:
birds_path = "/kaggle/input/birdclef-2022/scored_birds.json"
with open(birds_path) as bf:
    birds = json.load(bf)

In [ ]:
data = []
for f in files:
    file_path = test_path + f + '.ogg'
    audio, sr = librosa.load(file_path)
    buffer = 5 * sr
    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    while samples_wrote < samples_total:
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote          

        block = audio[samples_wrote : (samples_wrote + buffer)]
        feat = extractFeatures(block, sr)
        x = feat.reshape(1, num_rows, num_columns, num_channels)

        pred = cnn_model.predict(x)
        label_index = np.argmax(pred,axis=1)[0]
        
        for b in birds:
            segment_end = counter * 5 
            row_id = f + '_' + b + '_' + str(segment_end)
            target = False
            if labels[label_index] == b:
                target = True
            data.append([row_id, target])
        counter += 1
        samples_wrote += buffer
        
submission_df = pd.DataFrame(data, columns=['row_id', 'target'])
submission_df

In [ ]:
submission_df.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
shutil.rmtree("/kaggle/working/each5s")